In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

%matplotlib inline

In [2]:
service_requests = pd.read_csv('../data/hubNashville_311_service_requests.csv')
service_requests.head()

C:\Users\Owner\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,...,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
0,353977,Closed,COVID-19,COVID-19 - Submit Input about The Plan to Reop...,COVID-19 - Submit Input about The Plan to Reop...,04/28/2020 02:33:22 PM,04/28/2020 02:33:22 PM,hubNashville Community,NaN,NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,367021,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,05/13/2020 09:00:52 PM,05/13/2020 09:00:52 PM,Phone,NaN,NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,419194,Closed,"Streets, Roads & Sidewalks",Remove debris in roadway,Roadway,07/17/2020 03:21:03 PM,07/28/2020 04:06:19 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,Lafayette St & Claiborne St,NASHVILLE,17.0,37210,36.148412,-86.761179,"(36.1484116, -86.761179)"
3,366600,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,05/13/2020 04:04:14 PM,05/13/2020 04:04:14 PM,Phone,NaN,NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,408945,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,07/06/2020 05:57:28 PM,07/06/2020 05:57:28 PM,Phone,NaN,NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
service_requests['Request Type'].unique()

array(['COVID-19', 'Resolved by hubNashville on First Call',
       'Streets, Roads & Sidewalks', 'Trash, Recycling & Litter', 'Other',
       'Property Violations', 'Electric & Water General', 'Public Safety',
       'Storm Relief', 'Transit', 'Parks', 'Police Chief Search',
       'Permits', 'Social Services & Housing', nan, 'Planning & Zoning',
       'Hands On Volunteers', 'WIRES', 'Public Records Request',
       'Quick Requests', 'Workforce & Jobs', 'Education & Libraries',
       'BRUSH', 'Admin Support Request',
       'Submit Budget Ideas to Mayor Briley',
       'Consumer Protection Request'], dtype=object)

In [4]:
covid19 = service_requests[service_requests['Request Type'] == 'COVID-19']
covid19 = service_requests[service_requests['Subrequest Type'] == 'COVID-19 Violations']
covid19 = covid19.reset_index(drop = True)
covid19.head()

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,...,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
0,346122,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,04/15/2020 11:33:26 AM,04/16/2020 02:55:01 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,928 6th Ave S,NASHVILLE,17.0,37203,36.149040,-86.772273,"(36.1490396, -86.77227289999999)"
1,339374,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,04/04/2020 06:13:59 PM,04/09/2020 01:07:22 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,500 Gallatin Ave,NASHVILLE,5.0,37206,36.181548,-86.749695,"(36.181548, -86.749695)"
2,343513,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,04/10/2020 10:40:08 AM,04/14/2020 07:39:09 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,5720 Crossings Blvd,NaN,NaN,NaN,36.039608,-86.646541,"(36.039608, -86.646541)"
3,442996,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,08/12/2020 02:05:29 AM,08/12/2020 04:03:35 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,7689 Hwy 70 S,NASHVILLE,22.0,37221,36.078933,-86.953400,"(36.07893290000001, -86.9533998)"
4,339423,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,04/04/2020 08:12:27 PM,04/09/2020 02:21:09 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,5310 Mt View Rd,ANTIOCH,32.0,37013,36.048182,-86.648447,"(36.0481819, -86.6484474)"


In [5]:
covid19.shape

(11632, 21)

In [6]:
covid19['Additional Subrequest Type'].unique()

array(['COVID-19 Violations', nan], dtype=object)

In [7]:
covid19['System of Origin ID'].unique()

array([nan], dtype=object)

In [8]:
covid19['Contact Type'].unique()

array([nan, 'Resident', 'Other'], dtype=object)

In [9]:
covid19['Closed When Created'].value_counts()

False    11631
True         1
Name: Closed When Created, dtype: int64

In [10]:
covid19.columns

Index(['Request #', 'Status', 'Request Type', 'Subrequest Type',
       'Additional Subrequest Type', 'Date / Time Opened',
       'Date / Time Closed', 'Request Origin', 'System of Origin ID',
       'Contact Type', 'Preferred Language', 'State Issue', 'Parent Request',
       'Closed When Created', 'Address', 'City', 'Council District', 'ZIP',
       'Latitude', 'Longitude', 'Mapped Location'],
      dtype='object')

In [11]:
covid19 = covid19[['Request #', 'Date / Time Opened', 'Contact Type', 'Preferred Language', 'Closed When Created', 'Address', 'City', 'Council District', 'ZIP', 'Latitude', 'Longitude', 'Mapped Location']]
covid19.head()

,Request #,Date / Time Opened,Contact Type,Preferred Language,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
0,346122,04/15/2020 11:33:26 AM,NaN,NaN,False,928 6th Ave S,NASHVILLE,17.0,37203,36.149040,-86.772273,"(36.1490396, -86.77227289999999)"
1,339374,04/04/2020 06:13:59 PM,NaN,NaN,False,500 Gallatin Ave,NASHVILLE,5.0,37206,36.181548,-86.749695,"(36.181548, -86.749695)"
2,343513,04/10/2020 10:40:08 AM,NaN,NaN,False,5720 Crossings Blvd,NaN,NaN,NaN,36.039608,-86.646541,"(36.039608, -86.646541)"
3,442996,08/12/2020 02:05:29 AM,NaN,NaN,False,7689 Hwy 70 S,NASHVILLE,22.0,37221,36.078933,-86.953400,"(36.07893290000001, -86.9533998)"
4,339423,04/04/2020 08:12:27 PM,NaN,NaN,False,5310 Mt View Rd,ANTIOCH,32.0,37013,36.048182,-86.648447,"(36.0481819, -86.6484474)"


In [12]:
covid19.shape

(11632, 12)

In [14]:
covid19.to_csv('../data/covid_violations.csv')